# ECG feature extraction
### Process R-peak data into IBI and HRV features.
### Create a dataframe for LMM analysis in R.
#### 2024 by Antonin Fourcade

In [1]:
%matplotlib inline
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat

# Custom functions
from nvr_ecg_functions import castToList, heart_extract, NVR_ecg_features

# Parameters for plotting
plt.rcParams["figure.figsize"] = [20, 14]
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["xtick.labelsize"] = 20
plt.rcParams["ytick.labelsize"] = 20
plt.rcParams["legend.fontsize"] = 20
plt.rcParams["figure.autolayout"] = True

In [2]:
# Parameters
m_conds = ['mov', 'nomov'] # head movements conditions
a_conds = ['LA', 'HA'] # emotional arousal conditions
c_styles = 'SBA' # rollercoaster sequence: Space-Break-Andes
sampling_freq = 250 # ECG sampling frequency (Hz)
show_fig = 0; # 1: show IBI, LP and ratings plots for each SJ, 0: do not show

fs = 1 # sampling frequency for the ECG features: IBI, LF-HRV, HF-HRV
lf = [0.04, 0.15] # low frequency band for HRV, in Hz
hf = [0.15, 0.4] # high frequency band for HRV, in Hz

# length of the data in seconds to mirror at the beginning and at the end of the data
mirror_len=80
# length of the mirrored data in seconds to keep at the beginning and at the end of the data
cut=35

# Paths to the data and for saving
path_data = 'D:/NeVRo/Data/ECG/'
path_rpeak = path_data + 'R-peak_time_series/'
path_ibi = path_data + 'IBI/'
path_hrv = path_data + 'HRV/'
path_ar = path_data + '/ratings/class_bins/'

# Initialize the dictionary to store the data
ecg_features = {
    'mov': [],
    'nomov': []
}

# All mov are included in nomov, but nomov have more SJs
SJs_nomov = [2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 34, 36, 37, 39, 44]
SJs_mov = [2, 4, 5, 6, 8, 9, 11, 13, 14, 17, 18, 20, 21, 22, 24, 25, 27, 28, 29, 31, 34, 36, 37, 39, 44]

# Get the ECG features for each SJ and each mov condition

In [ ]:
# BE CAREFUL IT TAKES A LONG TIME (~2.5H)!
# Process R-peak data to compute Inter-Beat Intervals (IBI) and Heart Rate Variability (HRV) data.
for mc, m_cond in enumerate(m_conds):
    path_in_rpeak = path_rpeak + m_cond + '/'
    path_out_ibi = path_ibi + m_cond + '/'
    path_out_hrv = path_hrv + m_cond + '/'
    heart_extract(path_in_rpeak, path_out_ibi, path_out_hrv, m_cond, fs, lf=lf, hf=hf, mirror_len=mirror_len, cut=cut, fig=show_fig)

---------------------------------

In [ ]:
# Process data to extract ibi and hrv features 
# (e.g remove mirror padding, z-score, select HA and LA samples)
# Loop over the movement conditions:
for mc, m_cond in enumerate(m_conds):
    path_in_rpeak = path_rpeak + m_cond + '/'
    path_in_ibi = path_ibi + m_cond + '/'
    path_in_hrv = path_hrv + m_cond + '/'
    path_in_ar = path_ar + m_cond + '/' + c_styles + '/'
    ecg_features[m_cond] = NVR_ecg_features(path_in_rpeak, path_in_ibi, path_in_hrv, path_in_ar, cut=cut, fig=show_fig)        

In [ ]:
# Save ecg_features into file
filename_mat = 'ecg_features.mat'
savemat(path_data + filename_mat, ecg_features)

# Prepare data for Linear Mixed Modeling (LMM) in R

In [ ]:
# Load ecg_features from file
ecg_features = loadmat(path_data + 'ecg_features.mat', simplify_cells=True)

In [7]:
# Create a design dataframe for the statistical analysis
design = []
for i, subj in enumerate(SJs_nomov):
    if subj not in SJs_mov:
        mov_conds = ['nomov']
    else:
        mov_conds = m_conds
    for m in mov_conds:
        if m == 'mov':
            sj_idx = SJs_mov.index(subj)
        else:
            sj_idx = i
        for a in a_conds:
            ibi = castToList(ecg_features[m][sj_idx][a + '_ibi'])
            hrv_lf = castToList(ecg_features[m][sj_idx][a + '_hrv_lf'])
            hrv_hf = castToList(ecg_features[m][sj_idx][a + '_hrv_hf'])
            ratioLFHF = castToList(ecg_features[m][sj_idx][a + '_ratioLFHF'])
            #
            z_ibi = castToList(ecg_features[m][sj_idx][a + '_z_ibi'])
            z_hrv_lf = castToList(ecg_features[m][sj_idx][a + '_z_hrv_lf'])
            z_hrv_hf = castToList(ecg_features[m][sj_idx][a + '_z_hrv_hf'])
            z_ratioLFHF = castToList(ecg_features[m][sj_idx][a + '_z_ratioLFHF'])
            for s, s_ibi in enumerate(ibi):
                design.append([subj, m, a,
                               ibi[s], hrv_lf[s], hrv_hf[s], ratioLFHF[s],
                               z_ibi[s], z_hrv_lf[s], z_hrv_hf[s], z_ratioLFHF[s],
                               ])

In [8]:
# Convert into a pandas dataframe
design_df = pd.DataFrame(design, columns=['SJ', 'mov_cond', 'arousal',
                                          'ibi', 'hrv_lf', 'hrv_hf', 'ratio_lfhf',
                                          'z_ibi', 'z_hrv_lf', 'z_hrv_hf', 'z_ratio_lfhf',
                                          ])

In [10]:
# Save the design dataframe into a csv file
filename = path_data + 'nvr_ecg_features.csv'
design_df.to_csv(filename, index=False, na_rep ='NaN')